### Тестовое задание "Классификация цветков ириса"
<br><b>Выполнил:</b> <i>Альчаков В.В. гр. AD11872/3<i>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import tf2onnx
import onnx

from matplotlib import pyplot
%matplotlib inline

### Загрузка тестового набора

In [2]:
iris = pd.read_csv("iris.csv")
iris = iris.drop(['Id'], axis = 1)
iris = shuffle(iris)
iris

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
84,5.4,3.0,4.5,1.5,Iris-versicolor
39,5.1,3.4,1.5,0.2,Iris-setosa
105,7.6,3.0,6.6,2.1,Iris-virginica
51,6.4,3.2,4.5,1.5,Iris-versicolor
20,5.4,3.4,1.7,0.2,Iris-setosa
...,...,...,...,...,...
75,6.6,3.0,4.4,1.4,Iris-versicolor
34,4.9,3.1,1.5,0.1,Iris-setosa
139,6.9,3.1,5.4,2.1,Iris-virginica
144,6.7,3.3,5.7,2.5,Iris-virginica


### Кодирование признаков

In [3]:
lb = LabelEncoder()
lb.fit_transform(iris['Species'])

array([1, 0, 2, 1, 0, 0, 0, 1, 2, 1, 2, 0, 2, 2, 2, 1, 2, 1, 0, 2, 0, 0,
       0, 2, 2, 1, 1, 0, 2, 1, 2, 0, 2, 1, 1, 1, 2, 2, 0, 2, 2, 1, 1, 0,
       0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 1, 0, 0, 1, 1, 2,
       0, 2, 2, 2, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2,
       0, 1, 2, 0, 2, 1, 1, 2, 1, 2, 1, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 1,
       0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 0,
       0, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 2, 0, 1, 0, 2, 2, 1])

In [4]:
iris['Species'] = lb.fit_transform(iris['Species'])
iris

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
84,5.4,3.0,4.5,1.5,1
39,5.1,3.4,1.5,0.2,0
105,7.6,3.0,6.6,2.1,2
51,6.4,3.2,4.5,1.5,1
20,5.4,3.4,1.7,0.2,0
...,...,...,...,...,...
75,6.6,3.0,4.4,1.4,1
34,4.9,3.1,1.5,0.1,0
139,6.9,3.1,5.4,2.1,2
144,6.7,3.3,5.7,2.5,2


### Разбивка данных на тестовую и проверочную выборки

In [5]:
X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
y = pd.get_dummies(iris['Species']).values # One hot encoding
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100, shuffle = True)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


### Нормализация данных

In [6]:
normalizer = tf.keras.layers.Normalization(axis = -1)
normalizer.adapt(np.array(X))

### Определяем структуру нейронной сети

In [7]:
#model = tf.keras.Sequential([
#    tf.keras.layers.Dense(10, activation='relu'),
#    tf.keras.layers.Dense(10, activation='relu'),
#    tf.keras.layers.Dense(3, activation='softmax')
#])

n_features = X.shape[1] # Размерность X
model = Sequential()
#model.add(normalizer)
model.add(Dense(10, activation = 'relu', input_shape = (n_features, )))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [8]:
#model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 50, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/100
3/3 [==============================] - 0s 75ms/step - loss: 2.0654 - accuracy: 0.3583 - val_loss: 1.8945 - val_accuracy: 0.2333
Epoch 2/100
3/3 [==============================] - 0s 12ms/step - loss: 1.9859 - accuracy: 0.3583 - val_loss: 1.8236 - val_accuracy: 0.2333
Epoch 3/100
3/3 [==============================] - 0s 13ms/step - loss: 1.9122 - accuracy: 0.3583 - val_loss: 1.7555 - val_accuracy: 0.2333
Epoch 4/100
3/3 [==============================] - 0s 11ms/step - loss: 1.8380 - accuracy: 0.3583 - val_loss: 1.6914 - val_accuracy: 0.2333
Epoch 5/100
3/3 [==============================] - 0s 11ms/step - loss: 1.7626 - accuracy: 0.3583 - val_loss: 1.6310 - val_accuracy: 0.2333
Epoch 6/100
3/3 [==============================] - 0s 11ms/step - loss: 1.6993 - accuracy: 0.3667 - val_loss: 1.5716 - val_accuracy: 0.2333
Epoch 7/100
3/3 [==============================] - 0s 11ms/step - loss: 1.6362 - accuracy: 0.3833 - val_loss: 1.5140 - val_accuracy: 0.2333
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 11ms/step - loss: 0.5465 - accuracy: 0.7583 - val_loss: 0.6519 - val_accuracy: 0.7000
Epoch 60/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5420 - accuracy: 0.7583 - val_loss: 0.6473 - val_accuracy: 0.7000
Epoch 61/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5376 - accuracy: 0.7667 - val_loss: 0.6425 - val_accuracy: 0.7000
Epoch 62/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5334 - accuracy: 0.7667 - val_loss: 0.6378 - val_accuracy: 0.7000
Epoch 63/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5292 - accuracy: 0.7833 - val_loss: 0.6335 - val_accuracy: 0.7333
Epoch 64/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5251 - accuracy: 0.7833 - val_loss: 0.6288 - val_accuracy: 0.7333
Epoch 65/100
3/3 [==============================] - 0s 10ms/step - loss: 0.5212 - accuracy: 0.7833 - val_loss: 0.6242 - val_accuracy: 0.7667
Epoch 66/100
3/3 [========

In [10]:
loss, accuracy = model.evaluate(X_test, y_test, verbose = 0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.5075693130493164
Test accuracy: 0.8666666746139526


In [11]:
y_pred = model.predict(X_test)
y_pred

1/1 [==============================] - 0s 50ms/step


array([[0.15771835, 0.44802156, 0.39426008],
       [0.89277905, 0.09934256, 0.00787841],
       [0.22005224, 0.44624576, 0.333702  ],
       [0.12160403, 0.43770418, 0.4406918 ],
       [0.12936132, 0.46545166, 0.40518707],
       [0.20166971, 0.46024415, 0.3380861 ],
       [0.02424433, 0.28209874, 0.6936569 ],
       [0.92426175, 0.07131321, 0.00442502],
       [0.19859296, 0.45113912, 0.35026786],
       [0.01651254, 0.28522873, 0.6982587 ],
       [0.01948531, 0.28681177, 0.69370294],
       [0.02275234, 0.31006947, 0.66717815],
       [0.19369201, 0.47156963, 0.33473834],
       [0.01431829, 0.25989622, 0.72578543],
       [0.16984648, 0.47926205, 0.35089153],
       [0.03247847, 0.29258737, 0.67493415],
       [0.909719  , 0.08398234, 0.00629876],
       [0.01640357, 0.2743424 , 0.70925397],
       [0.12207516, 0.42552805, 0.45239678],
       [0.92098767, 0.07456632, 0.00444603],
       [0.01174393, 0.24595825, 0.7422978 ],
       [0.9375132 , 0.05932059, 0.00316631],
       [0.

In [12]:
actual = np.argmax(y_test, axis = 1)
predicted = np.argmax(y_pred, axis = 1)
print(f"Actual: {actual}")
print(f"Pred-d: {predicted}")

Actual: [1 0 1 1 1 1 2 0 1 2 2 2 1 2 1 2 0 2 1 0 2 0 0 1 2 1 1 0 1 1]
Pred-d: [1 0 1 2 1 1 2 0 1 2 2 2 1 2 1 2 0 2 2 0 2 0 0 2 2 2 1 0 1 1]


In [13]:
#onnx_model = keras2onnx.convert_keras(model, model.name)
#onnx.save_model(onnx_model, "iris.onnx")
# Экспорт модели
onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, "model\iris.onnx")